Describe how the equilibrium interest rate depends of the intertemporal discount factor,$\beta$, 
and intertemporal substitution/risk aversion, $\sigma$.

In [20]:
import numpy as np

In [21]:
def util_func(x):
    return (x**(1-sigma))/(1-sigma)

In [68]:
def parameters_objects():
    global beta, sigma,delta,alpha,pi,y_domain, a_domain, V, iG, n_y, n_a, w,r
    beta = 0.90
    sigma = 2
    delta = 0.08
    alpha = 0.44
    pi = np.array([[0.4 , 0.5 , 0.1], 
                   [0.3 , 0.2 , 0.5 ], 
                   [0.2 , 0.4 , 0.4]])
    r = 0.05 #we will need some interest rate to maximize our objective function
    w = 1 #we will need the wage too
    # the income domain was not defined we will crete one to make our program works properly 
    y_min = 1e-2
    y_max = 1
    n_y = len(pi) #must have one income for each state
    y_domain = np.linspace(y_min , y_max, n_y)
    # neither the assets domain was defined
    a_max = 10
    n_a = 9 #three assets distributions for each income state
    # we need a non-Ponzi condition for this case
    barA = 0 # no borrow allowed
    a_domain = np.linspace(-barA, a_max, n_a)
    
    #Now we just neeed some place to store our value function and policy function
    V = np.zeros((n_y, n_a)) #value
    iG = np.zeros((n_y, n_a), dtype = np.int)

In [23]:
def build_objective(V):
    global n_y, n_a, w, r
    F_OBJ = np.zeros((n_y, n_a, n_a)) #one dimension to each income, asset, and future asset
    #looping through all income, assets and future assets
    for i_y in range(n_y):
        y = y_domain[i_y]
        for i_a in range(n_a):
            a = a_domain[i_a]
            for i_a_line in range(n_a): #i_a_line stands for future assets
                aa = a_domain[i_a_line]
                c = w*y + a - aa/(1+r) #consumption 
                if c <= 0:
                    F_OBJ[i_y, i_a, i_a_line] = -np.inf
                else:
                    F_OBJ[i_y, i_a, i_a_line] = util_func(c) + beta*(np.dot(pi[i_y, :],V[:, i_a_line]))
    return F_OBJ

In [24]:
def maximize_TV_IG(F_OBJ):
    #maximizing for time t
    TV = np.zeros((n_y, n_a))
    T_iG = np.zeros((n_y, n_a), dtype = np.int)
    for i_y, y in enumerate(y_domain):
        for i_a, a in enumerate(a_domain):
            TV[i_y, i_a] = np.max(F_OBJ[i_y, i_a, :]) # max value of f_obj 
            T_iG[i_y, i_a] = np.argmax(F_OBJ[i_y, i_a, :]) # position associated to (y,a) pair that maximizes F_OBJ
    return TV, T_iG

In [25]:
def compute_V_G_est():
    global V
    norm, tol = 2, 1e-7
    while norm>tol:
        F_OBJ = build_objective(V)
        TV, T_iG = maximize_TV_IG(F_OBJ)
        norm = np.max(abs(TV - V))
        V = np.copy(TV)
        iG = np.copy(T_iG)
    return V, iG      

In [26]:
def compute_Q(iG):
    #This function gives a markov transition function, Q_{r} to compute, a stationary measure phi_{r}
    #associated associated to this transition function
    Q = np.zeros((n_y*n_a, n_y*n_a))
    for i_y in range(n_y):
        for i_a in range(n_a):
            c_state = i_y*n_a + i_a
            for i_y_line in range(n_y):
                for i_a_line in range(n_a):
                    n_state = i_y_line*n_a + i_a_line
                    if iG[i_y, i_a] == i_a_line:
                        Q[c_state, n_state] += pi[i_y, i_y_line]
    return Q

In [27]:
def compute_phi(Q):
    global phi
    phi = np.ones(n_y*n_a) / (n_y*n_a)
    norm_Q, tol_Q = 1, 1e-6
    while norm_Q > tol_Q:
        T_phi = np.dot(phi, Q)
        norm_Q = max(abs(T_phi - phi))
        phi = np.copy(T_phi)
    return phi

In [28]:
def compute_Ea(phi):
    Ea = 0
    for i_y in range(n_y):
        for i_a in range(n_a):
            s_index = iG[i_y, i_a] 
            savings = a_domain[s_index] 
            t_index = i_y * n_a + i_a
            size = phi[t_index]
            Ea += savings*size
    return Ea

In [29]:
def compute_L():
    L = 0
    for i_y in range(n_y):
        for i_a in range(n_a):
            labor_supply = y_domain[i_y]
            t_index = i_y*n_a + i_a
            size_l = phi[t_index]
            L += labor_supply*size_l
    return L

In [30]:
def compute_k(r):
    k =(alpha/(r+delta))**(1/(1-alpha))
    return k 

In [31]:
def compute_w(k):
    w = (1-alpha)*(k**alpha)
    return w

In [32]:
def compute_d(phi):
    k = compute_k(r)
    L = compute_L()
    K = k*L
    ea = compute_Ea(phi)
    d = K - ea
    return d

In [66]:
def compute_equilibrium():
    global r, w, V, iG, Q, phi, L, k
    rho = beta**(-1)-1
    r_1, r_2 = -delta, rho
    norm_r, tol_r = 1, 1e-10
    while norm_r>tol_r:
        r = (r_1+r_2)/2
        k = compute_k(r)
        V, iG = compute_V_G_est()
        Q = compute_Q(iG)    
        phi = compute_phi(Q)    
        d = compute_d(phi)
        if d>0:
            r_1 = r
        elif d<0:
            r_2 = r
        norm_r = abs(r_1-r_2)
        print('[d,r_L,r_H,norm]=[{:9.6f},{:9.6f},{:9.6f},{:9.6f}]'.format(d,r_1,r_2,norm_r))
        

In [69]:
parameters_objects()
compute_equilibrium()

O capital de equilíbrio  para essa taxa de juros é 15.285508234945167
[d,r_L,r_H,norm]=[ 5.200731, 0.015556, 0.111111, 0.095556]
O capital de equilíbrio  para essa taxa de juros é 7.410221785681968
[d,r_L,r_H,norm]=[-0.648122, 0.015556, 0.063333, 0.047778]
O capital de equilíbrio  para essa taxa de juros é 10.261864804092587
[d,r_L,r_H,norm]=[ 1.838470, 0.039444, 0.063333, 0.023889]
O capital de equilíbrio  para essa taxa de juros é 8.655871080723847
[d,r_L,r_H,norm]=[ 0.430048, 0.051389, 0.063333, 0.011944]
O capital de equilíbrio  para essa taxa de juros é 7.995346467855122
[d,r_L,r_H,norm]=[ 0.079274, 0.057361, 0.063333, 0.005972]
O capital de equilíbrio  para essa taxa de juros é 7.694114939130894
[d,r_L,r_H,norm]=[-0.080695, 0.057361, 0.060347, 0.002986]
O capital de equilíbrio  para essa taxa de juros é 7.8424750020861955
[d,r_L,r_H,norm]=[-0.001908, 0.057361, 0.058854, 0.001493]
O capital de equilíbrio  para essa taxa de juros é 7.918335257752005
[d,r_L,r_H,norm]=[ 0.038377, 0.0

Answer:

Let's see what happens with the equilibrium interest rate if we change sigma from 0.98 to 0.90. When $\beta = 0.98$,
the equilibrium interest rate is $2.04 % $, which is the same as $\rho$. Now, the agent became, impatient thus the his
respective beta became lower. That is, in order to the agent give up present consumption, he needs to be compensated,
in the intertemporal budget constraint context that means the equilibrium interest rate needs to be greater than 
before. So the respective interest rate equilibrium is about $5,9%$ which is a big increase compared to the variation 
of the betas. 
